In [1]:
import weave
import new_api
import openai
import time
import pandas as pd

In [2]:
@weave.op()
def predict(model_name, math, temp, trial):
    result = openai.chat.completions.create(model=model_name, messages=[{'role': 'user', 'content': math}])
    return result.choices[0].message.content

s = new_api.weave_sweep(predict,
                        # dataset
                        math=['1 + 2', '2 ** 64', 'x *** z'],
                        # model
                        model_name=['gpt-3.5-turbo', 'gpt-4o'],
                        temp=[0, 1],
                        # trials
                        trial=[0, 1])
print('Predicting')
results = s.compute()
df = pd.json_normalize([{**r[0], 'predict': r[1]} for r in results])
df.pivot(index=('math', 'trial'), columns=('model_name', 'temp'))

cache_hits: 0


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:22<00:00,  1.08it/s]


predict  \
model_name                                         gpt-3.5-turbo   
temp                                                           0   
math    trial                                                      
1 + 2   0                                                      3   
        1                                                      3   
2 ** 64 0                             18,446,744,073,709,551,616   
        1                             18,446,744,073,709,551,616   
x *** z 0      I'm sorry, could you provide more context or c...   
        1      I'm sorry, but I'm not sure what you're asking...   

                                                                  \
model_name                                                         
temp                                                           1   
math    trial                                                      
1 + 2   0                                                      3   
        1                                                      3   
2 ** 64 0                             18,446,744,073,709,551,616   
        1                                   18446744073709551616   
x *** z 0      I'm sorry, I'm not sure what you mean by "***"...   
        1      I'm sorry, I'm not sure what you're asking for...   

                                                                  \
model_name                                                gpt-4o   
temp                                                           0   
math    trial                                                      
1 + 2   0                                        1 + 2 equals 3.   
        1                                        1 + 2 equals 3.   
2 ** 64 0      The result of \(2^{64}\) is:\n\n\[ 2^{64} = 18...   
        1      2 raised to the power of 64 is equal to 18,446...   
x *** z 0      It looks like you have provided an expression ...   
        1      It looks like you're trying to use asterisks (...   

                                                                  
model_name                                                        
temp                                                           1  
math    trial                                                     
1 + 2   0                                        1 + 2 equals 3.  
        1                                        1 + 2 equals 3.  
2 ** 64 0      2 raised to the power of 64 is:\n\n2^64 = 18,4...  
        1      The expression \(2^{64}\) evaluates to the fol...  
x *** z 0      It looks like you have provided variables with...  
        1      It looks like you've used some asterisks betwe...

In [3]:
@weave.op()
def score(math, predict):
    prompt = f'Does this seem right to you? Respond with just NO or YES.\nQuestion:{math}\nAnswer:{predict}'
    result = openai.chat.completions.create(model='gpt-4o', messages=[{'role': 'user', 'content': prompt}])
    return result.choices[0].message.content

scores = new_api.weave_map(df, score)
scores_df = pd.json_normalize([{**r[0], 'score': r[1]} for r in scores])

cache_hits: 0


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:06<00:00,  2.45it/s]


In [8]:
# Join
pred_scores = df.merge(scores_df, on=('math', 'predict'))
pred_scores

,math,model_name,temp,trial,predict,score
0,1 + 2,gpt-3.5-turbo,0,0,3,YES
1,1 + 2,gpt-3.5-turbo,0,1,3,YES
2,1 + 2,gpt-3.5-turbo,1,0,3,YES
3,1 + 2,gpt-3.5-turbo,1,1,3,YES
4,1 + 2,gpt-4o,0,0,1 + 2 equals 3.,YES
5,1 + 2,gpt-4o,0,1,1 + 2 equals 3.,YES
6,1 + 2,gpt-4o,1,0,1 + 2 equals 3.,YES
7,1 + 2,gpt-4o,1,1,1 + 2 equals 3.,YES
8,2 ** 64,gpt-3.5-turbo,0,0,"18,446,744,073,709,551,616",YES
9,2 ** 64,gpt-3.5-turbo,0,1,"18,446,744,073,709,551,616",YES


In [5]:
pivoted = pred_scores.pivot(index=('math', 'trial'), columns=('model_name', 'temp'))
# Left rotate levels, so op is last.
pivoted.columns = pivoted.columns.reorder_levels((1, 2, 0))
pivoted = pivoted.sort_index(axis=1)
pivoted

model_name                                         gpt-3.5-turbo        \
temp                                                           0         
                                                         predict score   
math    trial                                                            
1 + 2   0                                                      3   YES   
        1                                                      3   YES   
2 ** 64 0                             18,446,744,073,709,551,616   YES   
        1                             18,446,744,073,709,551,616   YES   
x *** z 0      I'm sorry, could you provide more context or c...    NO   
        1      I'm sorry, but I'm not sure what you're asking...    NO   

model_name                                                              \
temp                                                           1         
                                                         predict score   
math    trial                                                            
1 + 2   0                                                      3   YES   
        1                                                      3   YES   
2 ** 64 0                             18,446,744,073,709,551,616   YES   
        1                                   18446744073709551616   YES   
x *** z 0      I'm sorry, I'm not sure what you mean by "***"...   YES   
        1      I'm sorry, I'm not sure what you're asking for...    NO   

model_name                                                gpt-4o        \
temp                                                           0         
                                                         predict score   
math    trial                                                            
1 + 2   0                                        1 + 2 equals 3.   YES   
        1                                        1 + 2 equals 3.   YES   
2 ** 64 0      The result of \(2^{64}\) is:\n\n\[ 2^{64} = 18...   YES   
        1      2 raised to the power of 64 is equal to 18,446...   YES   
x *** z 0      It looks like you have provided an expression ...    NO   
        1      It looks like you're trying to use asterisks (...    NO   

model_name                                                              
temp                                                           1        
                                                         predict score  
math    trial                                                           
1 + 2   0                                        1 + 2 equals 3.   YES  
        1                                        1 + 2 equals 3.   YES  
2 ** 64 0      2 raised to the power of 64 is:\n\n2^64 = 18,4...   YES  
        1      The expression \(2^{64}\) evaluates to the fol...   YES  
x *** z 0      It looks like you have provided variables with...    NO  
        1      It looks like you've used some asterisks betwe...    NO

In [6]:
# OK A few questions
# How do I use openai trials mode for batching?
# How do I add scorers?
# How do I make it work for prod data / embeddings, etc.

In [7]:
# Two options for scorers
# - Make a pipeline thing and pass that in for Op
# - What if I want to compute predictions in one pass, then feed to scorers in another?
# - Memoization does help with not needing that.
# - But still, feels natural

#weave.map(df, score)
# ... return with index from df?

# OK I think I'm close here.
# I want something that works both for evaluations and for production
# I think I'm missing a little part of the data model, because I need
# to be able to find all the scores for a given call
# I can do that now by memoization, as long as I know the table I'm in
# But I don't know the table I'm in
# Want to be able to add other records to calls

# OpCalls
#   - Columns
#     - categorize(inputs.message)
# This will work with memoization
#   But we have to know we want categorize
#   How do we also keep track of that categorize has been called on an op?
#   Like given an op, we want to see any consuming calls
#   Well that's what refs are for.
#   Seems maybe expensive?

# Question, if I do evals this new way, where is the record of what evals have been run?